# Homework 8
Billy Nayden

In [33]:
import pandas as pd
import numpy as np
import nltk
import textblob
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

np.set_printoptions(precision=2, linewidth=80)

In [34]:
# Read in spreadsheet with links
df = pd.read_excel('C:\\Users\\WilliamNayden\\Documents\\MSDS\\20201019_IMDB_Reviews_for_NLP.xlsx')

link_list = df['Link'].tolist()

## 1. In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.

In [35]:
#set up to remove stopwords
new_stops = list(stopwords.words('english'))

new_stops = new_stops + ['show', 'it', 'get', ' it.', 'it\'s', 'series', 'season', 
                         'can', 've', 'watch', 'watched', 'much', 'lot', 'thus', 'no', 'need', 'needed', 'should\'ve',
                        'it,', 'can.', 'can,']
stop_words = new_stops

In [36]:
#Append reviews to a list

output = []

#Append reviews to a list

for i in link_list[9:109]:
    lst = []
    
    #Parse HTML
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Pull out data we need
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    for i in span.split(' '):
            if i not in stop_words:
                lst.append(i)
    
    #Turn Reviews into List
    span_nostop = ' '.join(lst)
    
    output.append(str(span_nostop))

In [37]:
#Check output
print (output[1])

This feel like Star Wars. Now, I know people said viewing The Phantom Menace, prequels, despite flaws, delivered picture larger complex galaxy one introduced original trilogy. The prequel characters poorly written, scene direction felt times, Jar Jar existed, end day I'll take prequels new Disney films painted universe. They made want see Star Wars, I appreciated original trilogy's story arc even watching them. The Last Jedi complete opposite. It killed interest Star Wars universe seems nothing left care about. The Resistance Rebels. The First Order Empire. No end sight. Rey perfect already. Snoke dead one cared enough even explain existence first place. The original trilogy might well never happened. We've three death stars. No stable political structure seems possible galaxy, visit films. For know conflict could taking place one tiny sector galaxy rest galaxy completely loop. Sure, people space casino make money selling weapons slave kid Rebellion ring now, that's enough establish I 

In [38]:
#Get polarity score
sentiment_polarity = [textblob.TextBlob(review).sentiment.polarity for review in output]

In [39]:
#Categorize as positive or negative
sentiment_category = ['positive' if score >= 0.1  else 'negative' for score in sentiment_polarity]

In [40]:
#Convert to dataframe
df = pd.DataFrame(list(zip(output, sentiment_polarity, sentiment_category)),
              columns=['Review','Polarity', 'Category'])

df.head

<bound method NDFrame.head of                                                Review  Polarity  Category
0   I dont understand fan base anymore, people lik...  0.217686  positive
1   This feel like Star Wars. Now, I know people s...  0.086726  negative
2   The Last Jedi obsessed fallacious idea order m... -0.011543  negative
3   I relatively high expectations Episode VIII. U...  0.073099  negative
4   I hate TLJ, even I completely ignore (among ma...  0.045661  negative
..                                                ...       ...       ...
91  This movie, hyped movie time, put microscope l...  0.156279  positive
92  Well, waiting 16 years next installment,  Geor...  0.123223  positive
93  This movie begins back story anakin skywalker ...  0.232143  positive
94  I know fashionable scorn "prequel" trilogy, on...  0.177969  positive
95  The main line defence seems be: lighten up, en...  0.053697  negative

[96 rows x 3 columns]>

## 2.	For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.

In [41]:
#create vectorizer
vectorizer = TfidfVectorizer(stop_words={'english'})

In [42]:
#Perform tf-idf on reviews
links_vect = vectorizer.fit_transform(output)

In [43]:
#Look at shape
links_vect.shape

(96, 4800)

In [44]:
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
#https://scikit-learn.org/stable/modules/clustering.html#k-means
#https://matplotlib.org/tutorials/introductory/pyplot.html
#################################inertia_ is the sum of squared distance from cluster center#################################

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt #import matplotlib to examine intracluster distnaces

In [45]:
#Examine 7 Clusters
kmeans = KMeans(n_clusters=7, random_state=11)
clustered = kmeans.fit(links_vect)
clustered.labels_
#https://pythonprogramminglanguage.com/kmeans-text-clustering/

#Print top words of each cluster
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
words = vectorizer.get_feature_names()
for i in range(7):
    print("cluster:", i)
    for j in order_centroids[i, :10]:
        print(words[j])

cluster: 0
many
still
aspects
scrolling
heck
user
seen
stated
occasional
average
cluster: 1
anakin
the
movie
clones
wars
episode
attack
star
story
lucas
cluster: 2
movie
jar
the
good
ok
is
four
age
kid
talks
cluster: 3
movies
movie
star
wars
the
it
fan
original
liked
like
cluster: 4
even
good
bad
the
still
cgi
best
movie
film
worst
cluster: 5
the
film
star
and
force
it
rey
they
movie
luke
cluster: 6
luke
film
the
star
wars
this
empire
battle
one
love


In [46]:
#Check shape of cluster labels
print(clustered.labels_.shape)

(96,)


In [47]:
#Append cluster labels to dataframe
cluster_labels = clustered.labels_

df['Cluster'] = cluster_labels

df.head

<bound method NDFrame.head of                                                Review  Polarity  Category  \
0   I dont understand fan base anymore, people lik...  0.217686  positive   
1   This feel like Star Wars. Now, I know people s...  0.086726  negative   
2   The Last Jedi obsessed fallacious idea order m... -0.011543  negative   
3   I relatively high expectations Episode VIII. U...  0.073099  negative   
4   I hate TLJ, even I completely ignore (among ma...  0.045661  negative   
..                                                ...       ...       ...   
91  This movie, hyped movie time, put microscope l...  0.156279  positive   
92  Well, waiting 16 years next installment,  Geor...  0.123223  positive   
93  This movie begins back story anakin skywalker ...  0.232143  positive   
94  I know fashionable scorn "prequel" trilogy, on...  0.177969  positive   
95  The main line defence seems be: lighten up, en...  0.053697  negative   

    Cluster  
0         3  
1         5  
2  

In [48]:
#Compute mean polarity values
df.groupby('Cluster', as_index=False)['Polarity'].mean()

,Cluster,Polarity
0,0,0.067860
1,1,0.136171
2,2,0.038538
3,3,0.180502
4,4,0.124627
5,5,0.076720
6,6,0.242628


In [49]:
#Compute median polarity values
df.groupby('Cluster', as_index=False)['Polarity'].median()

,Cluster,Polarity
0,0,0.242857
1,1,0.138676
2,2,0.053697
3,3,0.202527
4,4,0.145588
5,5,0.074191
6,6,0.200558


In [50]:
#Compute max polarity values
df.groupby('Cluster', as_index=False)['Polarity'].max()

,Cluster,Polarity
0,0,0.344056
1,1,0.412188
2,2,0.156279
3,3,0.587946
4,4,1.000000
5,5,0.500000
6,6,1.000000


In [51]:
#Compute min polarity values
df.groupby('Cluster', as_index=False)['Polarity'].min()

,Cluster,Polarity
0,0,-0.383333
1,1,-0.169318
2,2,-0.152885
3,3,-0.404830
4,4,-0.766667
5,5,-0.388095
6,6,-0.350000


I think what's interesting is most of these reviews have positive polarity scores, indicating that many people enjoy Star Wars. I would be interested in viewing these scores by movie, using human created categories, rather the the clusters utilized here. I would imagine that the newer trilogies would have many more negative reviews than the original trilogies.

## 3.	For extra credit, analyze sentiment of chunks as follows:
   > a.	Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).

In [52]:
#NP Chunker from HW5

output = {}
counter = 1

np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

for i in link_list[9:109]:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    tokenized = word_tokenize(span)
    tagged = nltk.pos_tag(tokenized)
    chunked = np.parse(tagged)
    for subtree in chunked.subtrees():
        if subtree.label() == 'NP':
            lst.append(subtree)
    
    output["Review: " + str(counter)] = lst
    counter +=1

In [53]:
#Create an array of the chuncks
chunk_list = []
for k,v in output.items():
    chunk_list.append(v)

In [54]:
#Create a list we can extract the words from
chunks = []
for i in chunk_list:
    for j in i:
        chunks.append(j[0])

In [55]:
#Get words from chunks
tmp = []
for i in chunks:
    for j in i:
        tmp.append(j)

#List of just the words
words = tmp[::2]

In [56]:
#Get polarity scores
chunk_sent = []
for i in words:
    
    #Blob it
    blob = textblob.TextBlob(i)

    #Sentiment Analysis - this funciton removes punctuation and tokenizes 
    sent = blob.sentiment

    #add to sentiment list
    chunk_sent.append(sent.polarity)

In [57]:
#ensure the chunks and polarity scores are the same length
print(len(chunk_sent))
print(len(chunks))

7956
7956


In [58]:
chunk_sent_table = pd.DataFrame({'Chunk': chunks, 'Sentiment Score': chunk_sent})

### Sentiment/Chunk Table

In [59]:
chunk_sent_table

,Chunk,Sentiment Score
0,"(the, DT)",0.0000
1,"(most, JJS)",0.5000
2,"(Force, NNP)",0.0000
3,"(everyone, NN)",0.0000
4,"(Rogue, NNP)",0.0000
...,...,...
7951,"(Walt, NNP)",0.0000
7952,"(THAT, NNP)",0.0000
7953,"(Never, NN)",0.0000
7954,"(much, JJ)",0.2000


> b. Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see.

In [60]:
#Sort by highest sentiment score
cst_top = chunk_sent_table.sort_values(by='Sentiment Score', ascending=False).drop_duplicates()

cst_top.head(40)

,Chunk,Sentiment Score
5055,"(excellent, JJ)",1.000000
251,"(delicious, JJ)",1.000000
5801,"(Excellent, JJ)",1.000000
3855,"(wonderful, JJ)",1.000000
2992,"(best, JJS)",1.000000
2928,"(superb, NN)",1.000000
4914,"(perfect, JJ)",1.000000
2091,"(Legendary, JJ)",1.000000
1703,"(flawless, NN)",1.000000
6150,"(Flawless, NNP)",1.000000


In [61]:
#Sort by lowest sentiment score
cst_low = chunk_sent_table.sort_values(by='Sentiment Score', ascending=True).drop_duplicates()
cst_low.head(40)

,Chunk,Sentiment Score
903,"(terrible, JJ)",-1.00
6484,"(evil, JJ)",-1.00
460,"(cruel, JJ)",-1.00
3241,"(evil, NN)",-1.00
2248,"(insulting, JJ)",-1.00
3085,"(boring, JJ)",-1.00
6966,"(menacing, JJ)",-1.00
4746,"(awful, JJ)",-1.00
266,"(pathetic, JJ)",-1.00
4026,"(Worst, NNP)",-1.00


For the most part these sentiment scores make sense, but I think some of the proper nouns and character names in the reviews confuse the algorithm a bit. For example, the term "Rose" has one of the highest sentiment scores of any term, but refers to a character from the third Star Wars trilogy. Similarly "Grevious" has a very low sentiment score, but refers to General Greivous, a villain in the prequal trilogies. WHile both of these characters posses nomenclatures that reflect whether they are "good" or "bad" in the Star Wars universe, I find it interesting that this shows up in sentiment analysis.